Imports, huggingface login, cuda checking


In [ ]:
import transformers
import torch
import pandas as pd
from transformers import AutoTokenizer
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling
from transformers import (
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    EarlyStoppingCallback
)
import math
from huggingface_hub import login
from peft import LoraConfig, get_peft_model
from bitsandbytes.optim import Adam8bit
import os
import time
from torch.utils.data import DataLoader
import re
from pypdf import PdfReader
import json
from toolz import compose

print("hello world")

c:\Users\gsapountzakis\working_dir\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


hello world


In [ ]:

print(f"Torch version: {torch.__version__}")
print(torch.cuda.is_available())

print(torch.cuda.device_count())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Torch version: 2.5.1+cu124
True
1
cuda


Load Tokenizer and handle document


In [3]:
def remove_page_lines(text):
    return re.sub(
        r'\d+\s*Biotech\s*\(\d{4}\)\s*\d+:\d+\s*\d+\s*\d+\s*Page\s*\d+\s*of\s*\d+.*',
        "",
        text,
        flags=re.IGNORECASE,
    )

def remove_citations(text): #(numbers)
    return re.sub(
        r'\([^\)]*\d+[^\)]*\)',
        "",
        text,
        flags=re.IGNORECASE,
    )

def remove_declarations_and_references(text): 
    return re.sub(
        r'Declarations[^\n]*\n(?:[^\n]*\n)+',
        "",
        text,
        flags=re.IGNORECASE,
    )

def remove_intro(text):
    return re.sub(
        r'[^\n]*\n(?:[^\n]*\n)+Abstract',
        "",
        text,
        flags=re.IGNORECASE,
    )

def remove_keywords(text):
    return re.sub(
        r'Keywords[^\n]*\n(?:[^\n]*\n)',
        "",
        text,
        flags=re.IGNORECASE,
    )



In [4]:
clean_text_pipeline = compose(
    remove_citations,
    remove_page_lines,
    remove_declarations_and_references,
    remove_intro,
    remove_keywords
)

reader = PdfReader("Deep_learning_tools_for_advancing_drug_discovery.pdf")
text = ""
for page in reader.pages[1:19]:  
    page_text = page.extract_text()
    cleaned_page_text = clean_text_pipeline(page_text)
    text += cleaned_page_text + "\n\n"  # na to ksanado edo

text = re.sub(r'\.\n', '.\n\n\n', text)

with open("processed_output.txt", "w", encoding="utf-8") as file:
    file.write(text)

In [5]:
with open('processed_output_manual3.txt', 'r', encoding='utf-8') as file:
    text = file.read()

dataset_entries = [
    {"text": paragraph} for paragraph in text.split("\n\n") if paragraph.strip()
]
with open("dataset3.jsonl", "w", encoding="utf-8") as file:
    for entry in dataset_entries:
        file.write(json.dumps(entry) + "\n")




In [6]:

hf_dataset = Dataset.from_json("dataset3.jsonl")

print(hf_dataset)

Generating train split: 219 examples [00:00, 2605.97 examples/s]

Dataset({
    features: ['text'],
    num_rows: 219
})


In [7]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.model_max_length = 128
print("Loaded Tokenizer")

Loaded Tokenizer


In [8]:
def tokenize_function(data):
    return tokenizer(data["text"], truncation=True)

tokenized_data = hf_dataset.map(tokenize_function, batched=True, batch_size=32,num_proc=1,)
tokenized_data = tokenized_data.train_test_split(test_size=0.2)
print(tokenized_data)

Map: 100%|██████████| 219/219 [00:00<00:00, 3221.91 examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 175
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 44
    })
})


Load model and LoRA


In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct")#,quantization_config=bnb_config)
print(model)
print("Loaded Model")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb):

In [10]:
# # Freeze all parameters
# for param in model.parameters():
#     param.requires_grad = False


# for param in model.lm_head.parameters():#model.model.layers[12:15].parameters():
#     param.requires_grad = True



In [11]:
# # Calculate total and trainable parameters
# total_params = sum(p.numel() for p in model.parameters())
# trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# print(total_params)
# print(trainable_params)

# # Calculate the total size in bytes
# def model_size_in_gb(model):
#     total_size = 0
#     for param in model.parameters():
#         total_size += param.numel() * param.element_size()
#     for buffer in model.buffers():
#         total_size += buffer.numel() * buffer.element_size()
#     return total_size / (1024 ** 3)  # Convert bytes to gigabytes

# # Get model size on GPU
# gpu_size = model_size_in_gb(model)
# print(f"Model size on GPU: {gpu_size:.2f} GB")

In [12]:
# model = model.to(device)

In [ ]:
# If targeting all linear layers
target_modules = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "down_proj",
    "up_proj",
    "lm_head",
]

lora_config = LoraConfig(
    r=16,  # 8 16
    target_modules=target_modules,
    lora_alpha=8,  # 32 8
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)

lora_model.print_trainable_parameters()
print(lora_model)
print("Loaded Model with LoRA")

In [ ]:
# # Unfreeze the last N layers (e.g., last 2 transformer blocks)
# for name, param in lora_model.named_parameters():
#     if  name.split('.')[4]==('15' or '13' or '12' or '11' or '10')  and name.split('.')[5]==('self_attn' or 'mlp') :
#         param.requires_grad = True
#     print(name,param.requires_grad)


In [15]:
lora_model = lora_model.to(device)

Train save and evaluate model


In [16]:
training_args = TrainingArguments(
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    weight_decay=0.01,  # 0.01
    push_to_hub=False,
    output_dir="./results_lora_10",
    num_train_epochs=30,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=10,
    logging_dir="./logs",
    logging_steps=1000,
    save_steps=1000,
    save_total_limit=1,
    # deepspeed="./ds_config1.json",
    fp16=True,
    gradient_accumulation_steps=2,
    load_best_model_at_end = True
    #optim="paged_adamw_8bit"
)

In [17]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)  # ,pad_to_multiple_of=4
trainer = Trainer(
    model=lora_model, #model
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    data_collator=data_collator,
    processing_class=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [18]:
print("training started")
torch.cuda.empty_cache()
trainer.train()
print("training finished")

training started


Epoch,Training Loss,Validation Loss
1,No log,3.736198
2,No log,3.486528
3,No log,3.354820
4,No log,3.270620
5,No log,3.211726
6,No log,3.167627
7,No log,3.135380
8,No log,3.101021
9,No log,3.073367
10,No log,3.038384


c:\Users\gsapountzakis\working_dir\.venv\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
c:\Users\gsapountzakis\working_dir\.venv\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
c:\Users\gsapountzakis\working_dir\.venv\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
c:\Users\gsapountzakis\working_dir\.venv\Lib\site-packages\peft\utils\save_and_load.py:230: Us

training finished


In [19]:
trainer.save_model("./fine_tuned_model_lora_10")
print("saved trainer locally")

c:\Users\gsapountzakis\working_dir\.venv\Lib\site-packages\peft\utils\save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


saved trainer locally


In [20]:
eval_results = trainer.evaluate()
print(eval_results)
print("evaluation finished")

{'eval_loss': 2.9623308181762695, 'eval_runtime': 193.2627, 'eval_samples_per_second': 0.228, 'eval_steps_per_second': 0.228, 'epoch': 17.0}
evaluation finished


In [21]:
#lora = aplo
#lora 2 = lora
#lora 3 = unfreeze 12:15
#lora 4 = qlora
#lora 5 = many epochs lora
#lora 6 = many epochs lora + unfreeze
#lora 7 =  many epochs lora + unfreeze self_attn,mlp 10-15
#lora 8 =  many epochs lora + unfreeze self_attn,mlp 10-15 modified dataset
#lora 9 =  many epochs lora + unfreeze self_attn,mlp 10-15 modified dataset even more
#lora 10 =  many epochs lora + modified dataset even more
